### outline

- Descriptors are the machanism behind properties
- properties without descriptors
- custom descriptors
- data descriptors
- non-data descriptors
- attribute lookup

In [1]:
class Planet:
    def __init__(self, name, radius_meters, mass_kilograms, orbital_period_seconds, surface_temperature_kelvin):
        self.name = name
        self.radius_meters = radius_meters
        self.mass_kilograms = mass_kilograms
        self.orbital_period_seconds = orbital_period_seconds
        self.surface_temperature_kelvin = surface_temperature_kelvin

In [2]:
pluto = Planet(name='Pluto', radius_meters=1184e3, mass_kilograms=1.305e22, orbital_period_seconds=7816012992, surface_temperature_kelvin=55)

drawbacks: the amount of codes is explosive

In [3]:
class Planet:
    def __init__(self, name, radius_meters, mass_kilograms, orbital_period_seconds, surface_temperature_kelvin):
        self.name = name
        self.radius_meters = radius_meters
        self.mass_kilograms = mass_kilograms
        self.orbital_period_seconds = orbital_period_seconds
        self.surface_temperature_kelvin = surface_temperature_kelvin
    
    @property
    def name(self):
        return self._name
    
    @name.setter
    def name(self, val):
        if not val:
            raise ValueError("Cannot set an empty planet name!")
        self._name = val
        
    @property
    def radius_meters(self):
        return self._radius_meters
    
    @radius_meters.setter
    def radius_meters(self, val):
        if val <= 0:
            raise ValueError("Radius_meters value {} must be positive!".format(val))
        self._radius_meters = val

### Properties are descriptors

In [4]:
class Planet:
    def __init__(self, name, radius_meters, mass_kilograms, orbital_period_seconds, surface_temperature_kelvin):
        self.name = name
        self.radius_meters = radius_meters
        self.mass_kilograms = mass_kilograms
        self.orbital_period_seconds = orbital_period_seconds
        self.surface_temperature_kelvin = surface_temperature_kelvin
    

    def _get_name(self):
        return self._name
    
    def _set_name(self, val):
        if not val:
            raise ValueError("Cannot set an empty planet name!")
        self._name = val
    
    name = property(fget=_get_name, fset=_set_name)
    
    def _get_radius_meters(self):
        return self._radius_meters
    
    def _set_radius_meters(self, val):
        if val <= 0:
            raise ValueError("Radius_meters value {} must be positive!".format(val))
        self._radius_meters = val
    
    radius_meters = property(fget=_get_radius_meters, fset=_set_radius_meters)

In [5]:
pluto = Planet(name='Pluto', radius_meters=1184e3, mass_kilograms=1.305e22, orbital_period_seconds=7816012992, surface_temperature_kelvin=55)

In [6]:
print(pluto.radius_meters)

pluto.radius_meters = -50

1184000.0


ValueError: Radius_meters value -50 must be positive!

### Implement a descriptor

In [7]:
from weakref import WeakKeyDictionary

class Positive:
    def __init__(self):
        self._instance_data = WeakKeyDictionary()
        
    def __get__(self, instance, owner):
        return self._instance_data[instance]

    def __set__(self, instance, value):
        if value <= 0:
            raise ValueError("Value {} is not positive!".format(value))
        self._instance_data[instance] = value    
    
    def __del__(self, instance):
        raise AttributeError("Cannot delete attribute!")

In [8]:
class Planet:
    def __init__(self, name, radius_meters, mass_kilograms, orbital_period_seconds, surface_temperature_kelvin):
        self.name = name
        self.radius_meters = radius_meters
        self.mass_kilograms = mass_kilograms
        self.orbital_period_seconds = orbital_period_seconds
        self.surface_temperature_kelvin = surface_temperature_kelvin
    

    def _get_name(self):
        return self._name
    
    def _set_name(self, val):
        if not val:
            raise ValueError("Cannot set an empty planet name!")
        self._name = val
    
    name = property(fget=_get_name, fset=_set_name)
    
    radius_meters = Positive()

In [9]:
pluto = Planet(name='Pluto', radius_meters=1184e3, mass_kilograms=1.305e22, orbital_period_seconds=7816012992, surface_temperature_kelvin=55)

In [10]:
print(pluto.radius_meters)

pluto.radius_meters = -50

1184000.0


ValueError: Value -50 is not positive!

### Call descriptors from class

In [11]:
from weakref import WeakKeyDictionary

class Positive:
    def __init__(self):
        self._instance_data = WeakKeyDictionary()
        
    def __get__(self, instance, owner):
        if instance is None:
            return self
        return self._instance_data[instance]

    def __set__(self, instance, value):
        if value <= 0:
            raise ValueError("Value {} is not positive!".format(value))
        self._instance_data[instance] = value    
    
    def __del__(self, instance):
        raise AttributeError("Cannot delete attribute!")

In [12]:
class Planet:
    def __init__(self, name, radius_meters, mass_kilograms, orbital_period_seconds, surface_temperature_kelvin):
        self.name = name
        self.radius_meters = radius_meters
        self.mass_kilograms = mass_kilograms
        self.orbital_period_seconds = orbital_period_seconds
        self.surface_temperature_kelvin = surface_temperature_kelvin
    

    def _get_name(self):
        return self._name
    
    def _set_name(self, val):
        if not val:
            raise ValueError("Cannot set an empty planet name!")
        self._name = val
    
    name = property(fget=_get_name, fset=_set_name)
    
    radius_meters = Positive()

In [13]:
Planet.radius_meters

### class of descriptors

- Non-data descriptors define only \__get\__(); \__set\__() and \__del\__() are not defined
- Data descriptors support  \__get\__() and \__set\__() and are writable

**Attribute lookup precedence**
- If an instance \__dict\__ has an entry of the same name as a data descriptor, the data descriptor takes precedence
- If an instance \__dict\__ has an entry of the same name as a non-data descriptor, the \__dict\__ takes precedence

In [14]:
class DataDescriptor:
    def __get__(self, instance, owner):
        print("DataDescriptor.__get__({!r}, {!r}, {!r})".format(self, instance, owner))
    def __set__(self, instance, value):
        print("DataDescriptor.__set__({!r}, {!r}, {!r})".format(self, instance, value))
        
class NonDataDescriptor:
    def __get__(self, instance, owner):
        print("NonDataDescriptor.__get__({!r}, {!r}, {!r})".format(self, instance, owner))
        
class Owner:
    
    a = DataDescriptor()
    b = NonDataDescriptor()

In [15]:
obj = Owner()
obj.a

DataDescriptor.__get__(<__main__.DataDescriptor object at 0x0000020F72A05648>, <__main__.Owner object at 0x0000020F72A05208>, <class '__main__.Owner'>)


In [16]:
obj.__dict__['a'] = 196883
obj.a

DataDescriptor.__get__(<__main__.DataDescriptor object at 0x0000020F72A05648>, <__main__.Owner object at 0x0000020F72A05208>, <class '__main__.Owner'>)


In [17]:
obj.b

NonDataDescriptor.__get__(<__main__.NonDataDescriptor object at 0x0000020F72A05848>, <__main__.Owner object at 0x0000020F72A05208>, <class '__main__.Owner'>)


In [18]:
obj.__dict__['b'] = 740
obj.b

740